In [ ]:
# Do two steps: 
# 1: Export bounding boxes with YOLO: 

from pathlib import Path
import numpy as np
import cv2
from tqdm.notebook import tqdm
import torch
from ultralytics import YOLO
import shutil

CONFIDENCE_THRESHOLD = 0.15

# === Paths ===
video_folder = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO")
output_base = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO/DLC")

if output_base.exists():
    shutil.rmtree(output_base)
output_base.mkdir(parents=True)

video_files = list(video_folder.glob("*.mp4")) + list(video_folder.glob("*.MP4")) + \
              list(video_folder.glob("*.Mp4")) + list(video_folder.glob("*.mP4"))

print(f"🎥 Found {len(video_files)} video(s) to process.")
if not video_files:
    raise FileNotFoundError("No video files found!")

# === Load YOLOv11 model
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"⚡ Using device: {device}")
model = YOLO("yolo11l.pt")

# === Detection Loop ===
for i, video_path in enumerate(tqdm(video_files, desc="YOLO Detection", position=0)):
    print(f"\n📹 [{i+1}/{len(video_files)}] Processing: {video_path.name}")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"⚠️ Could not open video: {video_path}")
        continue

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_name = video_path.stem
    frame_output_dir = output_base / video_name / "frames"
    bbox_output_dir = output_base / video_name / "bboxes"
    frame_output_dir.mkdir(parents=True, exist_ok=True)
    bbox_output_dir.mkdir(parents=True, exist_ok=True)

    frame_idx = 0

    with tqdm(total=total_frames, desc="Processing frames", leave=False, position=1) as pbar:
        while True:
            ret, frame = cap.read()
            if not ret or frame is None:
                break

            # Save frame
            frame_path = frame_output_dir / f"{frame_idx:05d}.jpg"
            cv2.imwrite(str(frame_path), frame)

            # Run YOLOv8 detection (no classes filter here!)
            results = model.predict(frame, verbose=False, device=device, imgsz=640)

            # Manually filter for persons (class 0)
            bboxes_xywh = []
            for result in results:
                boxes = result.boxes
                if boxes is not None and boxes.xyxy is not None:
                    xyxy = boxes.xyxy.cpu().numpy()
                    cls = boxes.cls.cpu().numpy()
                    conf = boxes.conf.cpu().numpy()  # ← Add this line to get the confidences
                    for (x1, y1, x2, y2), label, confidence in zip(xyxy, cls, conf):
                        if int(label) == 0 and confidence >= CONFIDENCE_THRESHOLD:  # class 0 = person + confidence check
                            x = float(x1)
                            y = float(y1)
                            w = float(x2 - x1)
                            h = float(y2 - y1)
                            bboxes_xywh.append([x, y, w, h])

            bboxes_xywh = np.array(bboxes_xywh[:20], dtype=np.float32)

            if bboxes_xywh.size == 0:
                bboxes_xywh = np.empty((0, 4), dtype=np.float32)

            bbox_path = bbox_output_dir / f"{frame_idx:05d}.npy"
            np.save(str(bbox_path), bboxes_xywh)

            frame_idx += 1
            pbar.update(1)

    cap.release()
    print(f"✅ Finished: {video_name} with {frame_idx} frames processed.")

In [ ]:
# Step 2: RTM pose with DLC:

import deeplabcut.pose_estimation_pytorch as dlc_torch
from deeplabcut.utils.video_processor import VideoProcessorCV
from deeplabcut.utils.make_labeled_video import CreateVideo
import numpy as np
import torch
import cv2
import gc
from tqdm.notebook import tqdm
from pathlib import Path
import sys
import os
from contextlib import contextmanager
import deeplabcut.utils
deeplabcut.utils.tqdm = tqdm
import shutil

@contextmanager
def suppress_stdout():
    original_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    try:
        yield
    finally:
        sys.stdout.close()
        sys.stdout = original_stdout



# === Model Configuration Paths ===
path_model_config = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7_pytorch_config.yaml")
path_snapshot = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7.pt")
input_folder = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO/DLC") # Change the folder here!!!

# === Pose Model Settings ===
device = "mps"  # Use Apple Silicon MPS
pose_cfg = dlc_torch.config.read_config_as_dict(path_model_config)
runner = dlc_torch.get_pose_inference_runner(
    pose_cfg,
    snapshot_path=path_snapshot,
    batch_size=4,
    max_individuals=20,
    device=device
)

# === Load video directories ===
video_dirs = [d for d in input_folder.iterdir() if d.is_dir()]
print(f"📂 Found {len(video_dirs)} videos to process.")

# === Pose Estimation Loop ===
for video_dir in tqdm(video_dirs, desc="Pose Estimation", position=0):
    print(f"\n🧍‍♂️ Processing: {video_dir.name}")
    frame_dir = video_dir / "frames"
    bbox_dir = video_dir / "bboxes"

    frame_files = sorted(frame_dir.glob("*.jpg"))
    bbox_files = sorted(bbox_dir.glob("*.npy"))

    assert len(frame_files) == len(bbox_files), "Mismatch between frames and bbox files."

    output_csv_path = input_folder / f"{video_dir.name}_predictions.csv"
    partial_predictions = {}

    with tqdm(total=len(frame_files), desc="Pose estimation frames", leave=False, position=1) as pbar:
        for idx, (frame_file, bbox_file) in enumerate(zip(frame_files, bbox_files)):
            frame = cv2.imread(str(frame_file))
            if frame is None:
                print(f"⚠️ Failed to load frame: {frame_file}")
                continue

            bboxes = np.load(str(bbox_file), allow_pickle=True)
            frame_context = {"bboxes": bboxes}

            # Run inference on single frame
            pred = runner.inference([(frame, frame_context)])[0]
            partial_predictions[idx] = pred

            # Save every 100 frames
            if (idx + 1) % 100 == 0 or (idx + 1) == len(frame_files):
                df_partial = dlc_torch.build_predictions_dataframe(
                    scorer="rtmpose-body7",
                    predictions=partial_predictions,
                    parameters=dlc_torch.PoseDatasetParameters(
                        bodyparts=pose_cfg["metadata"]["bodyparts"],
                        unique_bpts=pose_cfg["metadata"]["unique_bodyparts"],
                        individuals=[f"idv_{i}" for i in range(20)]
                    )
                )
                df_partial.to_csv(output_csv_path)
                print(f"💾 Saved intermediate predictions at frame {idx+1}")
        
            pbar.update(1)

    print(f"✅ Finished pose estimation: {video_dir.name}")

    create_labeled_video = False  # Set this to False if you DON'T want labeled videos!!!

    # === Optional: Create labeled video IF NEEDED
    if create_labeled_video:
        original_video_path = Path("XXX") / f"{video_dir.name}.mp4"
        output_video_path = input_folder / f"{video_dir.name}_labeled.mp4"
    
        if original_video_path.exists():
            clip = VideoProcessorCV(str(original_video_path), sname=str(output_video_path), codec="mp4v")
            df_final = dlc_torch.build_predictions_dataframe(
                scorer="rtmpose-body7",
                predictions=partial_predictions,
                parameters=dlc_torch.PoseDatasetParameters(
                    bodyparts=pose_cfg["metadata"]["bodyparts"],
                    unique_bpts=pose_cfg["metadata"]["unique_bodyparts"],
                    individuals=[f"idv_{i}" for i in range(20)]
                )
            )
        
            print(f"🎬 Creating labeled video: {output_video_path.name}", end="", flush=True)
            
            with suppress_stdout():
                CreateVideo(
                    clip,
                    df_final,
                    pcutoff=0.4,
                    dotsize=5,
                    colormap="rainbow",
                    bodyparts2plot=pose_cfg["metadata"]["bodyparts"],
                    trailpoints=0,
                    cropping=False,
                    x1=0,
                    x2=clip.w,
                    y1=0,
                    y2=clip.h,
                    bodyparts2connect=[
                        [15, 13], [13, 11], [16, 14], [14, 12], [11, 12],
                        [5, 11], [6, 12], [5, 6], [5, 7], [6, 8],
                        [7, 9], [8, 10], [1, 2], [0, 1], [0, 2],
                        [1, 3], [2, 4], [3, 5], [4, 6]
                    ],
                    skeleton_color="k",
                    draw_skeleton=True,
                    displaycropped=False,
                    color_by="bodypart",
                )
            print(f"🎬 Labeled video saved: {output_video_path.name}")
        else:
            print(f"⚠️ Original video {original_video_path.name} not found, skipping labeled video.")
    
    # Continue cleanup regardless of the flag
    del partial_predictions
    torch.mps.empty_cache()
    gc.collect()

print("\n🎉 All pose estimations complete!")

# === REMOVE ALL DLC SUBFOLDERS (after all pose estimations are complete) ===
for subfolder in input_folder.iterdir():
    if subfolder.is_dir():
        try:
            shutil.rmtree(subfolder)
            print(f"🗑️ Deleted DLC subfolder: {subfolder}")
        except Exception as e:
            print(f"⚠️ Error deleting {subfolder}: {e}")

In [ ]:
# Step 3: REASSIGNMENT === UPDATED SCRIPT: Add Full Skeleton ===
# === FINAL CLEAN SCRIPT: Stable ID Assignment + Pose Matching + True Reassignment (Skeleton to Skeleton) ===

# === IMPORTS ===
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm

# === SETTINGS ===
MATCH_THRESHOLD_FRAME = 150          # Pixel-based threshold if not normalizing
USE_NORMALIZED_DISTANCE = True       # Switch for normalized matching, otherwise fixed distance matching
NORMALIZED_MATCH_THRESHOLD = 0.3     # Threshold if normalizing distances

# === LOAD VIDEO FILES ===
base_folder = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO")
video_files = list(base_folder.glob("*.mp4")) + list(base_folder.glob("*.MP4"))

# === MAIN LOOP: PROCESS EACH VIDEO ===
for video_path in video_files:
    print(f"\n🎥 Processing video: {video_path.name}")

    INPUT_PATH = video_path

    # === LOAD TRACKING DATA (Bounding Boxes) ===
    tracking_df = pd.read_csv(INPUT_PATH.parent / "YOLO" / (INPUT_PATH.stem + ".csv"))

    # === LOAD POSE ESTIMATION DATA (Skeletons) ===
    pose_df = pd.read_csv(INPUT_PATH.parent / "DLC" / (INPUT_PATH.stem + "_predictions.csv"), low_memory=False)

    # === PREPARE POSE DATAFRAME ===
    multi_index = pd.MultiIndex.from_arrays(pose_df.iloc[0:3].values, names=["individual", "bodypart", "coord"])
    pose_data_cleaned = pose_df.iloc[3:].copy()
    pose_data_cleaned.columns = multi_index
    pose_data_cleaned.reset_index(drop=True, inplace=True)
    pose_data_cleaned["Frame"] = pose_data_cleaned.index.astype(int)

    # === DETECT BODY PARTS (skip non-body columns) ===
    bodyparts = pose_data_cleaned.columns.get_level_values("bodypart").unique()
    bodyparts = [bp for bp in bodyparts if bp not in ["Frame", "scorer"]]

    # === FINAL DATA COLLECTOR ===
    final_data = []

    # === FRAME-BY-FRAME PROCESSING ===
    for frame_idx, pose_row in tqdm(pose_data_cleaned.iterrows(), total=len(pose_data_cleaned), desc=f"Processing {INPUT_PATH.stem}"):
        frame_num = int(pose_row["Frame"].iloc[0] if isinstance(pose_row["Frame"], pd.Series) else pose_row["Frame"])
        frame_boxes = tracking_df[tracking_df.Frame == frame_num]

        # === EXTRACT ALL SKELETON CENTERS IN THIS FRAME ===
        skeleton_centers = {}
        individuals = pose_row.index.get_level_values("individual").unique()
        individuals = [ind for ind in individuals if ind.startswith("idv_")]

        for ind in individuals:
            keypoints = []
            for bp in bodyparts:
                try:
                    x = float(pose_row[(ind, bp, "x")])
                    y = float(pose_row[(ind, bp, "y")])
                    if pd.notna(x) and pd.notna(y):
                        keypoints.append((x, y))
                except:
                    continue
            if keypoints:
                cx, cy = np.mean(keypoints, axis=0)  # Compute mean x, mean y = skeleton center
                skeleton_centers[ind] = (cx, cy)

        # === MATCH EACH BOUNDING BOX TO NEAREST SKELETON ===
        for _, box in frame_boxes.iterrows():
            x_center = (box.X1 + box.X2) / 2
            y_center = (box.Y1 + box.Y2) / 2
            pid = int(box.Person_ID)

            best_match = None
            min_distance = float('inf')

            for skel_id, (cx, cy) in skeleton_centers.items():
                center_distance = np.linalg.norm(np.array([cx, cy]) - np.array([x_center, y_center]))

                # === OPTIONAL: Normalize distance by bounding box size ===
                if USE_NORMALIZED_DISTANCE:
                    bbox_width = box.X2 - box.X1
                    bbox_height = box.Y2 - box.Y1
                    avg_bbox_size = (bbox_width + bbox_height) / 2
                    if avg_bbox_size > 0:
                        center_distance /= avg_bbox_size

                threshold = NORMALIZED_MATCH_THRESHOLD if USE_NORMALIZED_DISTANCE else MATCH_THRESHOLD_FRAME

                # === KEEP CLOSEST SKELETON UNDER THRESHOLD ===
                if center_distance < min_distance and center_distance < threshold:
                    best_match = skel_id
                    min_distance = center_distance

            # === SAVE MATCH INFORMATION ===
            data = {
                "Frame": frame_num,
                "Old_ID": pid,
                "New_ID": pid,
                "X1": box.X1,
                "Y1": box.Y1,
                "X2": box.X2,
                "Y2": box.Y2,
            }

            if best_match:
                skel_num = best_match.split("_")[1]
                for bp in bodyparts:
                    x = pose_row.get((f"idv_{skel_num}", bp, "x"), np.nan)
                    y = pose_row.get((f"idv_{skel_num}", bp, "y"), np.nan)
                    conf = pose_row.get((f"idv_{skel_num}", bp, "likelihood"), np.nan)
                    data[f"{bp}_x"] = x
                    data[f"{bp}_y"] = y
                    data[f"{bp}_conf"] = conf

            final_data.append(data)

    # === SAVE FINAL CORRECTED CSV ===
    final_df = pd.DataFrame(final_data)
    final_df = final_df.sort_values(["Frame", "New_ID"])
    final_df.drop(columns=["bodyparts_x", "bodyparts_y", "bodyparts_conf", "_x", "_y", "_conf"], errors="ignore", inplace=True)

    OUTPUT_FOLDER = INPUT_PATH.parent / "corrected"
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

    output_csv_name = f"{INPUT_PATH.stem}_corrected.csv"
    final_df.to_csv(OUTPUT_FOLDER / output_csv_name, index=False)

    print("\u2705 Final corrected CSV saved for:", video_path.name)

print("\n🎉 All sessions processed!")

In [ ]:
# Merge files: person classification and object classfication: 

import pandas as pd
from pathlib import Path

# === PATHS ===
main_folder = Path("/Users/Christian/Downloads/Javelin training/analysis/corrected")
object_folder = Path("/Users/Christian/Downloads/Javelin training/analysis/YOLO_Object")
merged_folder = main_folder / "with_objects"
merged_folder.mkdir(parents=True, exist_ok=True)

# === PROCESSING LOOP ===
for corrected_file in main_folder.glob("*_corrected.csv"):
    base_name = corrected_file.stem.replace("_corrected", "")
    object_file = object_folder / f"{base_name}.csv"

    if not object_file.exists():
        print(f"Skipping {base_name} — no object file found.")
        continue

    # Load both dataframes
    df_people = pd.read_csv(corrected_file)
    df_object = pd.read_csv(object_file)

    # Compute center points
    df_object["center_x"] = (df_object["X1"] + df_object["X2"]) / 2
    df_object["center_y"] = (df_object["Y1"] + df_object["Y2"]) / 2

    # Pivot: one row per frame, columns like Tip_center_x, Handle_center_y, etc.
    # Keep only needed columns
    df_object_slim = df_object[["Frame", "Label", "X1", "Y1", "X2", "Y2", "center_x", "center_y"]]
    
    # Pivot: one row per frame, columns like Tip_X1, Tip_center_x, etc.
    df_object_pivot = df_object_slim.pivot_table(
        index="Frame",
        columns="Label",
        aggfunc="first"
    )
    

    # Flatten multi-index columns
    df_object_pivot.columns = [f"{label}_{coord}" for coord, label in df_object_pivot.columns]
    df_object_pivot.reset_index(inplace=True)

    # Merge person + object center info
    df_merged = df_people.merge(df_object_pivot, on="Frame", how="left")

    # Save to new folder
    out_path = merged_folder / f"{base_name}_merged.csv"
    df_merged.to_csv(out_path, index=False)
    print(f"✅ Merged and saved: {out_path.name}")

print("\n🎉 All sessions processed!")

In [ ]:
# Just for visualization of how many were reassigned, not really necessary

import pandas as pd

# Load your final corrected file
csv_path = "/Users/Christian/Downloads/Fabrice Videos/1b/analysis/corrected/processed_1b mit Intervention_corrected.csv"
df = pd.read_csv(csv_path)

# Calculate basic statistics
total_frames = df['Frame'].nunique()
total_ids = len(df)
reassigned = df['Tried_Reassignment'].sum()
successful_reassigned = (df['Old_ID'] != df['New_ID']).sum()

# Print results
print(f"Total frames: {total_frames}")
print(f"Total ID entries: {total_ids}")
print(f"Reassignment attempts: {reassigned}")
print(f"Successful reassignments: {successful_reassigned}")
print(f"Reassignment attempt rate: {100 * reassigned / total_ids:.2f}% of all IDs")
print(f"Successful reassignment rate: {100 * successful_reassigned / total_ids:.2f}% of all IDs")

In [ ]:
# === UPDATED SCRIPT: FINAL CONFIRMATION VIDEO (for reassigned IDs and likelihoods) ===

# === FINAL CONFIRMATION VIDEOS (MULTIPLE FILES, ONLY FINAL_ID VISIBLE) ===

import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from tqdm.notebook import tqdm

# === SETTINGS ===
VIDEO_INPUT_FOLDER = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO")
FINAL_DF_INPUT_FOLDER = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO/corrected")
OUTPUT_FOLDER = Path("/Users/Christian/Downloads/Microadaptive Teaching Dritter Teil - LAs/Marlon/Erste Sitzung/YOLO/corrected")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)


ONLY_SHOW_HIGH_IDS = False  # Set to False to show all IDs (not only IDs > 1999)

# Target bodyparts list
target_bodyparts = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear",
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

# === PROCESS EACH FINAL CSV FILE ===
for final_csv in FINAL_DF_INPUT_FOLDER.glob("processed_*_corrected.csv"):
    session_name = final_csv.stem.replace("_corrected", "")
    print(f"\n🔵 Processing session: {session_name}")

    # Define corresponding video
    video_filename = f"{session_name}.MP4"
    video_path = VIDEO_INPUT_FOLDER / video_filename

    if not video_path.exists():
        print(f"⚠️ Video {video_filename} not found! Skipping.")
        continue

    # Load final corrected CSV
    final_df = pd.read_csv(final_csv)

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frame_limit = int(fps * 180)  # Only process first 30 seconds

    # Define output video path
    out_video_path = OUTPUT_FOLDER / f"visual_inspection_{session_name}.mp4"

    out_video = cv2.VideoWriter(
        str(out_video_path),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )

    colors = {}  # Final_ID → color mapping
    print(f"🎥 Creating video for {session_name}...")

    progress_bar = tqdm(total=min(total_frames, frame_limit), desc=f"Creating {session_name}", unit="frame", leave=True)

    right_wrist_start_y = {}
    
    right_wrist_start_y = {}

    for frame_idx in range(min(total_frames, frame_limit)):
        ret, frame = cap.read()
        if not ret:
            break
    
        frame_data = final_df[final_df.Frame == frame_idx]
    
        # Draw pink dot at object center (if available)
        if "Object_center_x" in frame_data.columns and not frame_data["Object_center_x"].isna().all():
            obj_centers = frame_data[["Object_center_x", "Object_center_y"]].dropna().drop_duplicates()
            for _, obj_row in obj_centers.iterrows():
                cx, cy = int(obj_row["Object_center_x"]), int(obj_row["Object_center_y"])
                cv2.circle(frame, (cx, cy), 24, (255, 105, 180), -1)  # pink dot
    
        for _, row in frame_data.iterrows():
            final_id = int(row["New_ID"])
    
            # Skip drawing if ONLY_SHOW_HIGH_IDS is True and ID <= 1999
            if ONLY_SHOW_HIGH_IDS and final_id <= 1999:
                continue
    
            color = colors.setdefault(final_id, tuple(np.random.randint(0, 255, size=3).tolist()))
    
            # Draw bounding box
            x1, y1, x2, y2 = int(row["X1"]), int(row["Y1"]), int(row["X2"]), int(row["Y2"])
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    
            # Draw ID label (ONLY Final_ID)
            x_center = (x1 + x2) // 2
            y_center = (y1 + y2) // 2
            id_label = f"ID {final_id}"
            cv2.putText(frame, id_label, (x_center + 10, y_center - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
    
            # ----- NEW: Calculate right leg length -----
            leg_length = np.nan
            if (not np.isnan(row["right_ankle_x"]) and not np.isnan(row["right_ankle_y"]) and
                not np.isnan(row["right_hip_x"]) and not np.isnan(row["right_hip_y"])):
                leg_length = np.sqrt(
                    (row["right_ankle_x"] - row["right_hip_x"])**2 +
                    (row["right_ankle_y"] - row["right_hip_y"])**2
                )
            if not np.isnan(leg_length):
                label_leg = f"Leg: {leg_length:.1f}"
                cv2.putText(frame, label_leg, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
            # ----- NEW: Calculate normalized change in right wrist y -----
            rw_y = row.get("right_wrist_y", np.nan)
            # Store the start y for each ID
            if final_id not in right_wrist_start_y and not np.isnan(rw_y):
                right_wrist_start_y[final_id] = rw_y
            delta_rw = np.nan
            if (final_id in right_wrist_start_y and not np.isnan(rw_y)
                and not np.isnan(leg_length) and leg_length > 1):
                delta_rw = (rw_y - right_wrist_start_y[final_id]) / leg_length
            if not np.isnan(delta_rw):
                label2 = f"Δwrist/leg: {delta_rw:+.2f}"
                cv2.putText(frame, label2, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 200, 0), 2)
    
            # Draw body markers
            keypoints = {}
            for part in target_bodyparts:
                x = row.get(f"{part}_x", np.nan)
                y = row.get(f"{part}_y", np.nan)
                conf = row.get(f"{part}_conf", np.nan)
    
                if not np.isnan(x) and not np.isnan(y) and (conf > 0.05):
                    keypoints[part] = (int(x), int(y))
                    cv2.circle(frame, (int(x), int(y)), 5, color, -1)
    
                    # Display the right wrist y-coordinate
                    if part == "right_wrist":
                        label = f"Y={int(y)}"
                        cv2.putText(
                            frame,
                            label,
                            (int(x) + 10, int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 255, 255),  # Yellow for visibility
                            2
                        )
    
            # Draw skeleton connections
            skeleton_connections = [
                ("left_shoulder", "right_shoulder"), ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
                ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
                ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
                ("left_hip", "right_hip"), ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
                ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
                ("nose", "left_eye"), ("nose", "right_eye"),
                ("left_eye", "left_ear"), ("right_eye", "right_ear")
            ]
    
            for bp1, bp2 in skeleton_connections:
                if bp1 in keypoints and bp2 in keypoints:
                    cv2.line(frame, keypoints[bp1], keypoints[bp2], color, 2)
    
        out_video.write(frame)
        progress_bar.update(1)

    cap.release()
    out_video.release()
    progress_bar.close()

    print(f"✅ Final video saved: {out_video_path.name}")

print("\n🎉 All sessions processed!")


In [ ]:
# STOP HERE!!!

In [ ]:
# ONLY FOR JAVELIN THROWING # === UPDATED SCRIPT: FINAL CONFIRMATION VIDEO (for reassigned IDs and likelihoods) ===

# === FINAL CONFIRMATION VIDEOS (MULTIPLE FILES, ONLY FINAL_ID VISIBLE) ===

import pandas as pd
import numpy as np
import cv2
from pathlib import Path
from tqdm.notebook import tqdm

# === SETTINGS ===
VIDEO_INPUT_FOLDER = Path("/Users/Christian/Downloads/Javelin training/analysis")
FINAL_DF_INPUT_FOLDER = Path("/Users/Christian/Downloads/Javelin training/analysis/corrected/with_objects")
OUTPUT_FOLDER = Path("/Users/Christian/Downloads/Javelin training/analysis/corrected/with_objects")
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

ONLY_SHOW_HIGH_IDS = False  # Set to False to show all IDs (not only IDs > 1999)

# Target bodyparts list
target_bodyparts = [
    "nose", "left_eye", "right_eye", "left_ear", "right_ear",
    "left_shoulder", "right_shoulder", "left_elbow", "right_elbow",
    "left_wrist", "right_wrist", "left_hip", "right_hip",
    "left_knee", "right_knee", "left_ankle", "right_ankle"
]

# === PROCESS EACH FINAL CSV FILE ===
for final_csv in FINAL_DF_INPUT_FOLDER.glob("*_merged.csv"):
    session_name = final_csv.stem.replace("_merged", "")
    print(f"\n🔵 Processing session: {session_name}")

    # Define corresponding video
    video_filename = f"{session_name}.MP4"
    video_path = VIDEO_INPUT_FOLDER / video_filename

    if not video_path.exists():
        print(f"⚠️ Video {video_filename} not found! Skipping.")
        continue

    # Load final corrected CSV
    final_df = pd.read_csv(final_csv)

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_limit = int(fps * 180)  # Only process first 30 seconds

    # Define output video path
    out_video_path = OUTPUT_FOLDER / f"visual_inspection_{session_name}.mp4"

    out_video = cv2.VideoWriter(
        str(out_video_path),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )

    colors = {}  # Final_ID → color mapping
    print(f"🎥 Creating video for {session_name}...")

    progress_bar = tqdm(total=min(total_frames, frame_limit), desc=f"Creating {session_name}", unit="frame", leave=True)

    print(f"📊 CSV frame range: {final_df['Frame'].min()} to {final_df['Frame'].max()}")
    print(f"🎞️ Video has {total_frames} frames")

    for frame_idx in range(min(total_frames, frame_limit)):
        ret, frame = cap.read()
        if not ret:
            break

        frame_data = final_df[final_df.Frame == frame_idx]


        # Draw Tip, Handle, and Tail object centers with distinct colors
        object_labels = ["Tip", "Handle", "Tail"]
        object_colors = {
            "Tip": (255, 105, 180),     # Pink
            "Handle": (0, 255, 0),      # Green
            "Tail": (0, 165, 255)       # Orange
        }
        
        for label in object_labels:
            x_col = f"{label}_center_x"
            y_col = f"{label}_center_y"
        
            if x_col in frame_data.columns and y_col in frame_data.columns:
                obj_coords = frame_data[[x_col, y_col]].dropna().drop_duplicates()
        
                for _, obj in obj_coords.iterrows():
                    cx, cy = int(obj[x_col]), int(obj[y_col])
                    cv2.circle(frame, (cx, cy), 20, object_colors[label], -1)
                    cv2.putText(frame, label, (cx + 10, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, object_colors[label], 2)


        for _, row in frame_data.iterrows():
            final_id = int(row["New_ID"])

            # Skip drawing if ONLY_SHOW_HIGH_IDS is True and ID <= 1999
            if ONLY_SHOW_HIGH_IDS and final_id <= 1999:
                continue
            
            color = (0, 0, 255)

            
            # Draw bounding box
            x1, y1, x2, y2 = int(row["X1"]), int(row["Y1"]), int(row["X2"]), int(row["Y2"])
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

            # Draw ID label (ONLY Final_ID)
            x_center = (x1 + x2) // 2
            y_center = (y1 + y2) // 2
            id_label = f"ID {final_id}"
            cv2.putText(frame, id_label, (x_center + 10, y_center - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)

            # Draw body markers
            keypoints = {}
            for part in target_bodyparts:
                x = row.get(f"{part}_x", np.nan)
                y = row.get(f"{part}_y", np.nan)
                conf = row.get(f"{part}_conf", np.nan)

                if not np.isnan(x) and not np.isnan(y) and (conf > 0.05):
                    keypoints[part] = (int(x), int(y))
                    cv2.circle(frame, (int(x), int(y)), 5, color, -1)

                    if part == "right_wrist":
                        label = f"Y={int(y)}"
                        cv2.putText(
                            frame,
                            label,
                            (int(x) + 10, int(y) - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            1,
                            (0, 255, 255),  # Yellow for visibility
                            2
                        )

            # Draw skeleton connections
            skeleton_connections = [
                ("left_shoulder", "right_shoulder"), ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
                ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
                ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
                ("left_hip", "right_hip"), ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
                ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
                ("nose", "left_eye"), ("nose", "right_eye"),
                ("left_eye", "left_ear"), ("right_eye", "right_ear")
            ]


            for bp1, bp2 in skeleton_connections:
                if bp1 in keypoints and bp2 in keypoints:
                    cv2.line(frame, keypoints[bp1], keypoints[bp2], color, 2)

        out_video.write(frame)
        progress_bar.update(1)

    cap.release()
    out_video.release()
    progress_bar.close()

    print(f"✅ Final video saved: {out_video_path.name}")

print("\n🎉 All sessions processed!")


In [ ]:
# Stop here: below are old code snippets

In [ ]:
# Step 3: REASSIGNMENT === UPDATED SCRIPT: Add Full Skeleton ===
# === FINAL CLEAN SCRIPT: Stable ID Assignment + Pose Matching + Old/New ID Tracking + Likelihoods ===

import pandas as pd
import numpy as np
import cv2
import os
from pathlib import Path
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

# === SIMPLE KALMAN FILTER CLASS ===
class SimpleKalman2D:
    def __init__(self, x, y):
        self.state = np.array([x, y, 0, 0], dtype=float)
        self.P = np.eye(4) * 1000
        self.F = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]])
        self.H = np.array([[1,0,0,0],[0,1,0,0]])
        self.R = np.eye(2) * 10
        self.Q = np.eye(4) * 0.01

    def predict(self):
        self.state = self.F @ self.state
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.state[0], self.state[1]

    def update(self, x, y):
        z = np.array([x, y])
        y_ = z - (self.H @ self.state)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.state += K @ y_
        self.P = (np.eye(4) - K @ self.H) @ self.P

# === SETTINGS ===
MATCH_THRESHOLD = 130
NORMALIZED_MATCH_THRESHOLD = 0.3  # for normalized mode
USE_NORMALIZED_DISTANCE = True    # <<< SWITCH: True = normalized, False = pixel distance

VIDEO_INPUT = "/Users/Christian/Downloads/Fabrice Videos/test/processed_1c ohne Intervention.MP4"
INPUT_PATH = Path(VIDEO_INPUT)
OUTPUT_FOLDER = INPUT_PATH.parent / "corrected"
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

# === LOAD DATA ===
tracking_df = pd.read_csv(INPUT_PATH.parent / "YOLO" / (INPUT_PATH.stem + ".csv"))
pose_df = pd.read_csv(INPUT_PATH.parent / "DLC" / (INPUT_PATH.stem + "_predictions.csv"), low_memory=False)

multi_index = pd.MultiIndex.from_arrays(pose_df.iloc[0:3].values, names=["individual", "bodypart", "coord"])
pose_data_cleaned = pose_df.iloc[3:].copy()
pose_data_cleaned.columns = multi_index
pose_data_cleaned.reset_index(drop=True, inplace=True)
pose_data_cleaned["Frame"] = pose_data_cleaned.index.astype(int)

bodyparts = pose_data_cleaned.columns.get_level_values("bodypart").unique()
bodyparts = [bp for bp in bodyparts if bp not in ["Frame", "scorer"]]

# === TRACKING STATE ===
kalman_filters = {}
previous_ids = set()

# === FINAL OUTPUT DATA ===
final_data = []

# === MAIN LOOP ===
for frame_idx, pose_row in tqdm(pose_data_cleaned.iterrows(), total=len(pose_data_cleaned), desc="Processing Frames"):
    frame_num = int(pose_row["Frame"])
    frame_boxes = tracking_df[tracking_df.Frame == frame_num]

    detected_ids = set(frame_boxes.Person_ID.values)
    individuals = pose_row.index.get_level_values("individual").unique()
    individuals = [ind for ind in individuals if ind.startswith("idv_")]

    # Prepare pose centroids
    pose_centroids = {}
    for ind in individuals:
        keypoints = []
        for bp in bodyparts:
            try:
                x = float(pose_row[(ind, bp, "x")])
                y = float(pose_row[(ind, bp, "y")])
                if pd.notna(x) and pd.notna(y):
                    keypoints.append((x, y))
            except:
                continue
        if keypoints:
            cx, cy = np.mean(keypoints, axis=0)
            pose_centroids[ind] = (cx, cy)

    # Detect if reassignment is necessary
    lost_ids = previous_ids - detected_ids
    new_ids = detected_ids - previous_ids

    # Build reassignment map
    reassignment_map = {}
    tried_reassignment_flags = {}

    for new_id in new_ids:
        box = frame_boxes[frame_boxes.Person_ID == new_id].iloc[0]
        x_center = (box.X1 + box.X2) / 2
        y_center = (box.Y1 + box.Y2) / 2

        best_match = None
        min_distance = float('inf')

        for skel_id, (cx, cy) in pose_centroids.items():
            # Calculate center distance (always)
            center_distance = np.sqrt((cx - x_center)**2 + (cy - y_center)**2)
            
            # Normalize distance if needed
            if USE_NORMALIZED_DISTANCE:
                bbox_width = box.X2 - box.X1
                bbox_height = box.Y2 - box.Y1
                avg_bbox_size = (bbox_width + bbox_height) / 2
                if avg_bbox_size > 0:
                    center_distance /= avg_bbox_size
            
            # Check match threshold
            threshold = NORMALIZED_MATCH_THRESHOLD if USE_NORMALIZED_DISTANCE else MATCH_THRESHOLD
            
            if center_distance < min_distance and center_distance < threshold:
                best_match = skel_id
                min_distance = center_distance


        if best_match is not None:
            reassignment_map[new_id] = new_id  # keep ID
            tried_reassignment_flags[new_id] = 1
        else:
            reassignment_map[new_id] = new_id  # no match, accept new ID
            tried_reassignment_flags[new_id] = 1

    # IDs that are consistent (no reassignment needed)
    for stable_id in detected_ids.intersection(previous_ids):
        reassignment_map[stable_id] = stable_id
        tried_reassignment_flags[stable_id] = 0

    # === Save frame info ===
    for pid in detected_ids:
        box = frame_boxes[frame_boxes.Person_ID == pid].iloc[0]
        new_id = reassignment_map.get(pid, pid)

        data = {
            "Frame": frame_num,
            "Old_ID": pid,
            "New_ID": new_id,
            "Tried_Reassignment": tried_reassignment_flags.get(pid, 0),
            "X1": box.X1,
            "Y1": box.Y1,
            "X2": box.X2,
            "Y2": box.Y2,
        }

        # Attach skeleton if match possible
        skel_num = None
        for skel_id, (cx, cy) in pose_centroids.items():
            center_box = ((box.X1 + box.X2) / 2, (box.Y1 + box.Y2) / 2)
            
            center_distance = np.linalg.norm(np.array([cx, cy]) - np.array(center_box))
        
            if USE_NORMALIZED_DISTANCE:
                bbox_width = box.X2 - box.X1
                bbox_height = box.Y2 - box.Y1
                avg_bbox_size = (bbox_width + bbox_height) / 2
                if avg_bbox_size > 0:
                    center_distance /= avg_bbox_size
        
            threshold = NORMALIZED_MATCH_THRESHOLD if USE_NORMALIZED_DISTANCE else MATCH_THRESHOLD
        
            if center_distance < threshold:
                skel_num = skel_id.split("_")[1]
                break
        

        if skel_num:
            for bp in bodyparts:
                x = pose_row.get((f"idv_{skel_num}", bp, "x"), np.nan)
                y = pose_row.get((f"idv_{skel_num}", bp, "y"), np.nan)
                conf = pose_row.get((f"idv_{skel_num}", bp, "likelihood"), np.nan)
                data[f"{bp}_x"] = x
                data[f"{bp}_y"] = y
                data[f"{bp}_conf"] = conf

        final_data.append(data)

    previous_ids = detected_ids.copy()

# === FINALIZE ===
final_df = pd.DataFrame(final_data)
final_df = final_df.sort_values(["Frame", "Old_ID"])
final_df.to_csv(OUTPUT_FOLDER / "final_corrected_output_stable_with_old_new_and_likelihoods.csv", index=False)

print("\u2705 Final corrected CSV with Old/New IDs, reassignment flags, and body markers saved!")



In [ ]:
# Reassignment based on YOLO, FastReID, and RTM: 

import pandas as pd
import numpy as np
import cv2
import os
from pathlib import Path
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

# === SETTINGS ===
MATCH_THRESHOLD = 100
MOVEMENT_THRESHOLD = 50
VIDEO_INPUT = "/Users/Christian/Downloads/Fabrice Videos/test/processed_1c ohne Intervention.MP4"
input_video_path = Path(VIDEO_INPUT)
OUTPUT_FOLDER = input_video_path.parent / "corrected"
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)



bodypart_colors = {
    "nose": (255, 0, 0),       # red
    "left_eye": (0, 255, 0),    # green
    "right_eye": (0, 0, 255),   # blue
    "left_ear": (255, 255, 0),  # cyan
    "right_ear": (255, 0, 255), # magenta
    "left_shoulder": (0, 255, 255),
    "right_shoulder": (255, 128, 0),
    "left_elbow": (128, 0, 255),
    "right_elbow": (0, 128, 255),
    "left_wrist": (128, 128, 0),
    "right_wrist": (0, 128, 128),
    "left_hip": (255, 128, 128),
    "right_hip": (128, 255, 128),
    "left_knee": (128, 128, 255),
    "right_knee": (255, 255, 128),
    "left_ankle": (255, 128, 255),
    "right_ankle": (128, 255, 255),
}

# === STEP 1: Load data ===
tracking_df = pd.read_csv("/Users/Christian/Downloads/Fabrice Videos/test/YOLO/processed_1c ohne Intervention.csv")
pose_df = pd.read_csv("/Users/Christian/Downloads/Fabrice Videos/test/DLC/processed_1c ohne Intervention_predictions.csv", low_memory=False)

# === STEP 2: Prepare pose data ===
multi_index = pd.MultiIndex.from_arrays(pose_df.iloc[0:3].values, names=["individual", "bodypart", "coord"])
pose_data_cleaned = pose_df.iloc[3:].copy()
pose_data_cleaned.columns = multi_index
pose_data_cleaned.reset_index(drop=True, inplace=True)
pose_data_cleaned["Frame"] = pose_data_cleaned.index.astype(int)

bodyparts = pose_data_cleaned.columns.get_level_values("bodypart").unique()
bodyparts = [bp for bp in bodyparts if bp not in ["Frame", "scorer"]]

# === STEP 3: Match skeletons to bounding boxes ===
tracking_by_frame = tracking_df.groupby("Frame")
enriched_matches = []

for frame_idx, row in tqdm(pose_data_cleaned.iterrows(), total=len(pose_data_cleaned), desc="Matching Skeletons"):
    frame_num = int(row["Frame"].iloc[0])
    if frame_num not in tracking_by_frame.groups:
        continue

    frame_boxes = tracking_by_frame.get_group(frame_num)
    individuals = pose_data_cleaned.columns.get_level_values("individual").unique()
    individuals = [ind for ind in individuals if ind.startswith("idv_")]

    skeleton_centroids = []
    skeleton_ids = []

    for ind in individuals:
        keypoints = []
        for bp in bodyparts:
            try:
                x = float(row[(ind, bp, "x")])
                y = float(row[(ind, bp, "y")])
                if pd.notna(x) and pd.notna(y):
                    keypoints.append((x, y))
            except (ValueError, KeyError, TypeError):
                continue

        if not keypoints:
            continue

        cx, cy = np.mean(keypoints, axis=0)
        skeleton_centroids.append((cx, cy))
        skeleton_ids.append(ind)

    if not skeleton_centroids:
        continue

    skeleton_centroids = np.array(skeleton_centroids)
    box_centers = np.column_stack(((frame_boxes.X1 + frame_boxes.X2) / 2, (frame_boxes.Y1 + frame_boxes.Y2) / 2))
    distances = cdist(skeleton_centroids, box_centers)

    for sk_idx, sk_id in enumerate(skeleton_ids):
        min_dist = np.min(distances[sk_idx])
        min_idx = np.argmin(distances[sk_idx])
        if min_dist < MATCH_THRESHOLD:
            matched_person_id = frame_boxes.iloc[min_idx].Person_ID
            enriched_matches.append({
                "Frame": frame_num,
                "Skeleton_ID": sk_id,
                "Centroid_X": skeleton_centroids[sk_idx][0],
                "Centroid_Y": skeleton_centroids[sk_idx][1],
                "Matched_Person_ID": matched_person_id,
                "Distance": min_dist
            })

enriched_df = pd.DataFrame(enriched_matches)

# === STEP 4: Track and Correct Skeleton Trajectories ===
skeleton_tracks = {}
for sk_id, group in enriched_df.groupby("Skeleton_ID"):
    trajectory = group.sort_values("Frame")[["Frame", "Centroid_X", "Centroid_Y", "Matched_Person_ID"]].reset_index(drop=True)
    skeleton_tracks[sk_id] = trajectory

corrections = []
for sk_id, df in skeleton_tracks.items():
    previous_id = None
    for idx, row in df.iterrows():
        current_id = row["Matched_Person_ID"]
        frame = row["Frame"]
        if previous_id is not None and current_id != previous_id:
            prev_row = df.iloc[idx - 1]
            dx = row["Centroid_X"] - prev_row["Centroid_X"]
            dy = row["Centroid_Y"] - prev_row["Centroid_Y"]
            movement = np.sqrt(dx ** 2 + dy ** 2)
            if movement < 50:
                corrections.append({
                    "Frame": frame,
                    "Skeleton_ID": sk_id,
                    "Old_ID": current_id,
                    "New_ID": previous_id
                })
                df.at[idx, "Matched_Person_ID"] = previous_id
        previous_id = df.at[idx, "Matched_Person_ID"]

corrected_df = pd.concat(skeleton_tracks.values(), ignore_index=True)

# Add bounding box info
corrected_with_boxes = pd.merge(
    corrected_df,
    tracking_df.rename(columns={"Person_ID": "Matched_Person_ID"}),
    on=["Frame", "Matched_Person_ID"],
    how="left"
)

# Flatten pose markers
pose_export = pose_data_cleaned.copy()
pose_export_frame = pose_export["Frame"]
pose_export.columns = ['_'.join(col).strip() for col in pose_export.columns.values]
pose_export["Frame"] = pose_export_frame  # restore usable 'Frame' key
final_output = pd.merge(corrected_with_boxes, pose_export, on="Frame", how="left")

final_output.to_csv(Path(OUTPUT_FOLDER) / "corrected_with_boxes_and_pose.csv", index=False)

# === STEP 5: Create Visual Inspection Video ===
cap = cv2.VideoCapture(VIDEO_INPUT)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out_video = cv2.VideoWriter(str(Path(OUTPUT_FOLDER) / "visual_inspection.mp4"),
                            cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

colors = {}
print("Creating video...")

for frame_idx in tqdm(range(total_frames), desc="Creating Video"):
    ret, frame = cap.read()
    if not ret:
        break

    frame_matches = corrected_df[corrected_df.Frame == frame_idx]
    frame_boxes = tracking_df[tracking_df.Frame == frame_idx]
    pose_row = pose_data_cleaned.loc[pose_data_cleaned["Frame"] == frame_idx]

    for _, row in frame_matches.iterrows():
        pid = int(row["Matched_Person_ID"])
        cx, cy = int(row["Centroid_X"]), int(row["Centroid_Y"])
        color = colors.setdefault(pid, tuple(np.random.randint(0, 255, size=3).tolist()))
        color = bodypart_colors.get(bp, (255, 255, 0))  # fallback to yellow
        cv2.circle(frame, (int(x), int(y)), 5, color, -1)
        cv2.putText(frame, f'ID {pid}', (cx + 10, cy - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)

    for _, box in frame_boxes.iterrows():
        x1, y1, x2, y2 = int(box.X1), int(box.Y1), int(box.X2), int(box.Y2)
        pid = int(box.Person_ID)
        color = colors.setdefault(pid, tuple(np.random.randint(0, 255, size=3).tolist()))
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

    # draw all bodymarkers per person
    if not pose_row.empty:
        individuals = pose_row.columns.get_level_values("individual").unique()
        individuals = [ind for ind in individuals if ind.startswith("idv_")]
        for ind in individuals:
            for bp in bodyparts:
                try:
                    x = float(pose_row[(ind, bp, "x")].values[0])
                    y = float(pose_row[(ind, bp, "y")].values[0])
                    if pd.notna(x) and pd.notna(y):
                        cv2.circle(frame, (int(x), int(y)), 3, (255, 255, 0), -1)
                except:
                    continue

    out_video.write(frame)

cap.release()
out_video.release()
print("✅ Visual inspection video created!")

In [ ]:
# DLC and RMT Pose:
# https://huggingface.co/DeepLabCut/HumanBody/tree/main

from pathlib import Path
import deeplabcut.pose_estimation_pytorch as dlc_torch
from deeplabcut.utils.video_processor import VideoProcessorCV
from deeplabcut.utils.make_labeled_video import CreateVideo
import cv2
import torchvision.models.detection as detection
from PIL import Image
import numpy as np
import torch
from tqdm.notebook import tqdm
import shutil
import gc

# === INPUT & OUTPUT PATHS ===
video_folder = Path("/Users/Christian/Downloads/Fabrice Videos/test")  # Folder with input .mp4 videos
output_dir = Path("/Users/Christian/Downloads/Fabrice Videos/test/DLC")    # Folder to save outputs

# Clean and recreate output folder
if output_dir.exists():
    shutil.rmtree(output_dir)
output_dir.mkdir(parents=True)

# Load all video files
video_files = list(video_folder.glob("*.mp4")) + list(video_folder.glob("*.MP4")) + list(video_folder.glob("*.Mp4")) + list(video_folder.glob("*.mP4"))

# === Model Configuration Paths ===
path_model_config = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7_pytorch_config.yaml")
path_snapshot = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7.pt")

# === Inference Settings ===
device = "cpu"  # change to "cuda" if using GPU
max_detections = 20

# === Load Object Detector ===
weights = detection.FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT
detector = detection.fasterrcnn_mobilenet_v3_large_fpn(weights=weights, box_score_thresh=0.6)
detector.eval().to(device)
preprocess = weights.transforms()

# === Load RTMPose Model ===
pose_cfg = dlc_torch.config.read_config_as_dict(path_model_config)
runner = dlc_torch.get_pose_inference_runner(
    pose_cfg,
    snapshot_path=path_snapshot,
    batch_size=4,
    max_individuals=max_detections,
    device=device
)

# === Process Each Video ===
for video_path in tqdm(video_files, desc="Processing videos"):
    print(f"\n📹 Processing: {video_path.name}")

    output_csv_path = output_dir / f"{video_path.stem}_predictions.csv"
    output_video_path = output_dir / f"{video_path.stem}_labeled.mp4"

    # === Extract Frames ===
    cap = cv2.VideoCapture(str(video_path))
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    # === Run Object Detection ===
    context = []
    with torch.no_grad():
        for frame in tqdm(frames, desc="Running object detection", leave=False):
            image = Image.fromarray(frame).convert("RGB")
            batch = [preprocess(image).to(device)]
            predictions = detector(batch)[0]
            bboxes = predictions["boxes"].cpu().numpy()
            labels = predictions["labels"].cpu().numpy()

            # Keep only 'person' detections (label 1)
            human_bboxes = [bbox for bbox, label in zip(bboxes, labels) if label == 1]
            if human_bboxes:
                # Use largest bbox (by area)
                human_bboxes = sorted(human_bboxes, key=lambda b: (b[2]-b[0])*(b[3]-b[1]), reverse=True)
                bbox = human_bboxes[0]
                bbox[2] -= bbox[0]  # convert to width
                bbox[3] -= bbox[1]  # convert to height
                bboxes = np.array([bbox])
            else:
                bboxes = np.zeros((0, 4))

            context.append({"bboxes": bboxes})

    # === Run RTMPose Inference ===
    frames_with_context = list(zip(frames, context))
    predictions = runner.inference(tqdm(frames_with_context, desc="Running RTMPose", leave=False))

    # === Save Predictions ===
    df = dlc_torch.build_predictions_dataframe(
        scorer="rtmpose-body7",
        predictions={i: pred for i, pred in enumerate(predictions)},
        parameters=dlc_torch.PoseDatasetParameters(
            bodyparts=pose_cfg["metadata"]["bodyparts"],
            unique_bpts=pose_cfg["metadata"]["unique_bodyparts"],
            individuals=[f"idv_{i}" for i in range(max_detections)]
        )
    )
    df.to_csv(output_csv_path)
    print(f"✅ Saved predictions: {output_csv_path.name}")

    # === Create Labeled Video ===
    clip = VideoProcessorCV(str(video_path), sname=str(output_video_path), codec="mp4v")
    CreateVideo(
        clip,
        df,
        pcutoff=0.4,
        dotsize=3,
        colormap="rainbow",
        bodyparts2plot=pose_cfg["metadata"]["bodyparts"],
        trailpoints=0,
        cropping=False,
        x1=0,
        x2=clip.w,
        y1=0,
        y2=clip.h,
        bodyparts2connect=[
            [15, 13], [13, 11], [16, 14], [14, 12], [11, 12],
            [5, 11], [6, 12], [5, 6], [5, 7], [6, 8],
            [7, 9], [8, 10], [1, 2], [0, 1], [0, 2],
            [1, 3], [2, 4], [3, 5], [4, 6]
        ],
        skeleton_color="k",
        draw_skeleton=True,
        displaycropped=False,
        color_by="bodypart",
    )
    print(f"🎬 Labeled video saved: {output_video_path.name}")

    # === Clear memory
    del frames
    del context
    del frames_with_context
    del predictions
    del df
    torch.cuda.empty_cache()
    gc.collect()

print("\n✅ All videos processed and saved to:", output_dir)

In [ ]:
# RTM Pose only for Pictures:
# https://huggingface.co/DeepLabCut/HumanBody/tree/main

from pathlib import Path
import deeplabcut.pose_estimation_pytorch as dlc_torch
import matplotlib.pyplot as plt
import matplotlib.collections as collections
import torchvision.models.detection as detection
from PIL import Image
import numpy as np
import torch
from tqdm.notebook import tqdm

# === Absolute paths to your files ===
image_folder = Path("/Users/Christian/rtm_pose/images")
path_model_config = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7_pytorch_config.yaml")
path_snapshot = Path("/Users/Christian/rtm_pose/rtmpose-x_simcc-body7.pt")
output_csv_path = Path("/Users/Christian/rtm_pose/image_predictions.csv")

# === Device and config ===
device = "cpu"  # Use "mps" if you want to try the Apple GPU
max_detections = 1

# === Load object detector ===
weights = detection.FasterRCNN_MobileNet_V3_Large_FPN_Weights.DEFAULT
detector = detection.fasterrcnn_mobilenet_v3_large_fpn(
    weights=weights,
    box_score_thresh=0.6,
)
detector.eval().to(device)
preprocess = weights.transforms()

# === Detect people in images ===
image_paths = sorted(image_folder.glob("*.jpg"))
context = []

print("Running object detection...")
with torch.no_grad():
    for image_path in tqdm(image_paths, leave=True):
        image = Image.open(image_path).convert("RGB")
        batch = [preprocess(image).to(device)]
        predictions = detector(batch)[0]
        bboxes = predictions["boxes"].cpu().numpy()
        labels = predictions["labels"].cpu().numpy()

        human_bboxes = [bbox for bbox, label in zip(bboxes, labels) if label == 1]

        bboxes = np.zeros((0, 4))
        if human_bboxes:
            bboxes = np.stack(human_bboxes)
        bboxes[:, 2] -= bboxes[:, 0]
        bboxes[:, 3] -= bboxes[:, 1]
        bboxes = bboxes[:max_detections]

        context.append({"bboxes": bboxes})

# === Run RTMPose inference ===
pose_cfg = dlc_torch.config.read_config_as_dict(path_model_config)
runner = dlc_torch.get_pose_inference_runner(
    pose_cfg,
    snapshot_path=path_snapshot,
    batch_size=4,
    max_individuals=max_detections,
)

print("Running pose estimation...")
predictions = runner.inference(zip(image_paths, context))

# === Save predictions to CSV ===
print("Saving predictions...")
df = dlc_torch.build_predictions_dataframe(
    scorer="rtmpose-body7",
    predictions={str(p): pred for p, pred in zip(image_paths, predictions)},
    parameters=dlc_torch.PoseDatasetParameters(
        bodyparts=pose_cfg["metadata"]["bodyparts"],
        unique_bpts=pose_cfg["metadata"]["unique_bodyparts"],
        individuals=[f"idv_{i}" for i in range(max_detections)]
    )
)
df.to_csv(output_csv_path)
print(f"✅ Done! Predictions saved to: {output_csv_path}")

In [ ]:
# Save labelled pictures: 

# === Skeleton definition ===
skeleton = [
    [16, 14], [14, 12], [17, 15], [15, 13],
    [12, 13], [6, 12], [7, 13], [6, 7],
    [6, 8], [7, 9], [8, 10], [9, 11],
    [2, 3], [1, 2], [1, 3], [2, 4],
    [3, 5], [4, 6], [5, 7],
]

# === Plot options ===
plot_skeleton = True
plot_pose_markers = True
plot_bounding_boxes = True
marker_size = 12
cmap_keypoints = plt.get_cmap("rainbow")
cmap_skeleton = plt.get_cmap("rainbow")

# === Folder to save visualizations ===
output_dir = Path("/Users/Christian/rtm_pose/labeled_images")
output_dir.mkdir(parents=True, exist_ok=True)

# === Loop through and save images ===
for image_path, image_predictions in zip(image_paths, predictions):
    image = Image.open(image_path).convert("RGB")
    pose = image_predictions["bodyparts"]
    bboxes = image_predictions["bboxes"]
    num_individuals, num_bodyparts = pose.shape[:2]

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image)
    ax.set_xlim(0, image.width)
    ax.set_ylim(image.height, 0)
    ax.axis("off")

    for idv_pose in pose:
        if plot_skeleton:
            bones = []
            for bpt_1, bpt_2 in skeleton:
                bones.append([idv_pose[bpt_1 - 1, :2], idv_pose[bpt_2 - 1, :2]])
            bone_colors = cmap_skeleton(np.linspace(0, 1, len(skeleton)))
            ax.add_collection(collections.LineCollection(bones, colors=bone_colors))

        if plot_pose_markers:
            ax.scatter(
                idv_pose[:, 0],
                idv_pose[:, 1],
                c=list(range(num_bodyparts)),
                cmap=cmap_keypoints,
                s=marker_size,
            )

    if plot_bounding_boxes:
        for x, y, w, h in bboxes:
            ax.plot([x, x + w, x + w, x, x], [y, y, y + h, y + h, y], c="red")

    # === Save image ===
    output_file = output_dir / f"{image_path.stem}_labeled.jpg"
    plt.savefig(output_file, bbox_inches="tight", pad_inches=0.1)
    plt.close(fig)  # Close the figure to avoid memory buildup

    print(f"✅ Saved: {output_file}")

In [ ]:
# Show labelled pictures:

# === Skeleton definition (same as in the model config) ===
skeleton = [
    [16, 14], [14, 12], [17, 15], [15, 13],
    [12, 13], [6, 12], [7, 13], [6, 7],
    [6, 8], [7, 9], [8, 10], [9, 11],
    [2, 3], [1, 2], [1, 3], [2, 4],
    [3, 5], [4, 6], [5, 7],
]

# === Plot options ===
plot_skeleton = True
plot_pose_markers = True
plot_bounding_boxes = True
marker_size = 12
cmap_keypoints = plt.get_cmap("rainbow")
cmap_skeleton = plt.get_cmap("rainbow")

# === Loop through images and predictions ===
for image_path, image_predictions in zip(image_paths, predictions):
    image = Image.open(image_path).convert("RGB")

    pose = image_predictions["bodyparts"]
    bboxes = image_predictions["bboxes"]
    num_individuals, num_bodyparts = pose.shape[:2]

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.imshow(image)
    ax.set_xlim(0, image.width)
    ax.set_ylim(image.height, 0)
    ax.axis("off")

    for idv_pose in pose:
        if plot_skeleton:
            bones = []
            for bpt_1, bpt_2 in skeleton:
                bones.append([idv_pose[bpt_1 - 1, :2], idv_pose[bpt_2 - 1, :2]])

            bone_colors = cmap_skeleton(np.linspace(0, 1, len(skeleton)))
            ax.add_collection(collections.LineCollection(bones, colors=bone_colors))

        if plot_pose_markers:
            ax.scatter(
                idv_pose[:, 0],
                idv_pose[:, 1],
                c=list(range(num_bodyparts)),
                cmap=cmap_keypoints,
                s=marker_size,
            )

    if plot_bounding_boxes:
        for x, y, w, h in bboxes:
            ax.plot([x, x + w, x + w, x, x], [y, y, y + h, y + h, y], c="red")

    plt.title(image_path.name)
    plt.show()

In [ ]:
# Step 3: REASSIGNMENT === UPDATED SCRIPT: Add Full Skeleton + Kalman Filter ===

import pandas as pd
import numpy as np
import cv2
import os
from pathlib import Path
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm

# === SIMPLE KALMAN FILTER CLASS ===
class SimpleKalman2D:
    def __init__(self, x, y):
        self.state = np.array([x, y, 0, 0], dtype=float)
        self.P = np.eye(4) * 1000
        self.F = np.array([[1,0,1,0],[0,1,0,1],[0,0,1,0],[0,0,0,1]])
        self.H = np.array([[1,0,0,0],[0,1,0,0]])
        self.R = np.eye(2) * 10
        self.Q = np.eye(4) * 0.01

    def predict(self):
        self.state = self.F @ self.state
        self.P = self.F @ self.P @ self.F.T + self.Q
        return self.state[0], self.state[1]

    def update(self, x, y):
        z = np.array([x, y])
        y_ = z - (self.H @ self.state)
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.state += K @ y_
        self.P = (np.eye(4) - K @ self.H) @ self.P

# === SETTINGS ===
MATCH_THRESHOLD = 100
MOVEMENT_THRESHOLD = 50
VIDEO_INPUT = "/Users/Christian/Downloads/Fabrice Videos/test/processed_1c ohne Intervention.MP4"
input_video_path = Path(VIDEO_INPUT)
OUTPUT_FOLDER = input_video_path.parent / "corrected"
OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

kalman_filters = {}

# === BODY PART COLORS ===
bodypart_colors = {
    "nose": (255, 0, 0), "left_eye": (0, 255, 0), "right_eye": (0, 0, 255),
    "left_ear": (255, 255, 0), "right_ear": (255, 0, 255),
    "left_shoulder": (0, 255, 255), "right_shoulder": (255, 128, 0),
    "left_elbow": (128, 0, 255), "right_elbow": (0, 128, 255),
    "left_wrist": (128, 128, 0), "right_wrist": (0, 128, 128),
    "left_hip": (255, 128, 128), "right_hip": (128, 255, 128),
    "left_knee": (128, 128, 255), "right_knee": (255, 255, 128),
    "left_ankle": (255, 128, 255), "right_ankle": (128, 255, 255),
}

# === LOAD DATA ===
tracking_df = pd.read_csv(input_video_path.parent / "YOLO" / (input_video_path.stem + ".csv"))
pose_df = pd.read_csv(input_video_path.parent / "DLC" / (input_video_path.stem + "_predictions.csv"), low_memory=False)

multi_index = pd.MultiIndex.from_arrays(pose_df.iloc[0:3].values, names=["individual", "bodypart", "coord"])
pose_data_cleaned = pose_df.iloc[3:].copy()
pose_data_cleaned.columns = multi_index
pose_data_cleaned.reset_index(drop=True, inplace=True)
pose_data_cleaned["Frame"] = pose_data_cleaned.index.astype(int)

bodyparts = pose_data_cleaned.columns.get_level_values("bodypart").unique()
bodyparts = [bp for bp in bodyparts if bp not in ["Frame", "scorer"]]

# === MATCH SKELETONS TO BOXES ===
tracking_by_frame = tracking_df.groupby("Frame")
enriched_matches = []

for frame_idx, row in tqdm(pose_data_cleaned.iterrows(), total=len(pose_data_cleaned), desc="Matching Skeletons"):
    frame_num = int(row["Frame"].item())
    if frame_num not in tracking_by_frame.groups:
        continue

    frame_boxes = tracking_by_frame.get_group(frame_num)
    individuals = pose_data_cleaned.columns.get_level_values("individual").unique()
    individuals = [ind for ind in individuals if ind.startswith("idv_")]

    skeleton_centroids = []
    skeleton_ids = []

    for ind in individuals:
        keypoints = []
        for bp in bodyparts:
            try:
                x = float(row[(ind, bp, "x")])
                y = float(row[(ind, bp, "y")])
                if pd.notna(x) and pd.notna(y):
                    keypoints.append((x, y))
            except:
                continue

        if not keypoints:
            continue

        cx, cy = np.mean(keypoints, axis=0)

        # Kalman filter tracking
        if ind not in kalman_filters:
            kalman_filters[ind] = SimpleKalman2D(cx, cy)
        else:
            kalman_filters[ind].update(cx, cy)
            cx, cy = kalman_filters[ind].predict()

        skeleton_centroids.append((cx, cy))
        skeleton_ids.append(ind)

    if not skeleton_centroids:
        continue

    skeleton_centroids = np.array(skeleton_centroids)
    box_centers = np.column_stack(((frame_boxes.X1 + frame_boxes.X2) / 2, (frame_boxes.Y1 + frame_boxes.Y2) / 2))
    distances = cdist(skeleton_centroids, box_centers)

    for sk_idx, sk_id in enumerate(skeleton_ids):
        min_dist = np.min(distances[sk_idx])
        min_idx = np.argmin(distances[sk_idx])
        if min_dist < MATCH_THRESHOLD:
            matched_person_id = frame_boxes.iloc[min_idx].Person_ID
            enriched_matches.append({
                "Frame": frame_num,
                "Skeleton_ID": sk_id,
                "Centroid_X": skeleton_centroids[sk_idx][0],
                "Centroid_Y": skeleton_centroids[sk_idx][1],
                "Matched_Person_ID": matched_person_id,
                "Distance": min_dist
            })

# === CORRECTION ===
enriched_df = pd.DataFrame(enriched_matches)
skeleton_tracks = {}
for sk_id, group in enriched_df.groupby("Skeleton_ID"):
    trajectory = group.sort_values("Frame")[["Frame", "Centroid_X", "Centroid_Y", "Matched_Person_ID"]].reset_index(drop=True)
    skeleton_tracks[sk_id] = trajectory

for sk_id, df in skeleton_tracks.items():
    previous_id = None
    for idx, row in df.iterrows():
        current_id = row["Matched_Person_ID"]
        frame = row["Frame"]
        if previous_id is not None and current_id != previous_id:
            prev_row = df.iloc[idx - 1]
            dx = row["Centroid_X"] - prev_row["Centroid_X"]
            dy = row["Centroid_Y"] - prev_row["Centroid_Y"]
            movement = np.sqrt(dx ** 2 + dy ** 2)
            if movement < MOVEMENT_THRESHOLD:
                df.at[idx, "Matched_Person_ID"] = previous_id
        previous_id = df.at[idx, "Matched_Person_ID"]

corrected_df = pd.concat(skeleton_tracks.values(), ignore_index=True)

# === MERGE WITH POSE & TRACKING ===
corrected_with_boxes = pd.merge(
    enriched_df,
    tracking_df.rename(columns={"Person_ID": "Matched_Person_ID"}),
    on=["Frame", "Matched_Person_ID"],
    how="left"
)

pose_export = pose_data_cleaned.copy()
pose_export_frame = pose_export["Frame"]
pose_export.columns = ['_'.join(col).strip() for col in pose_export.columns.values]
pose_export["Frame"] = pose_export_frame
final_output = pd.merge(corrected_with_boxes, pose_export, on="Frame", how="left")
final_output.to_csv(Path(OUTPUT_FOLDER) / "corrected_with_boxes_and_pose.csv", index=False)

print("✅ Correction complete with full skeleton and Kalman filtering.")



# === Create Visual Inspection Video ===
create_labeled_video = False  # Set this to False if you don't want labeled videos

# === Optional: Create labeled video if needed
if create_labeled_video:
    cap = cv2.VideoCapture(str(VIDEO_INPUT))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    out_video = cv2.VideoWriter(
        str(OUTPUT_FOLDER / "visual_inspection.mp4"),
        cv2.VideoWriter_fourcc(*'mp4v'),
        fps,
        (frame_width, frame_height)
    )
    
    colors = {}
    print("🎥 Creating visualization video...")
    
    progress_bar = tqdm(total=total_frames, desc="Creating Video", unit="frame", ncols=300, leave=True)
    
    for frame_idx in tqdm(range(total_frames), desc="Creating Video"):
        ret, frame = cap.read()
        if not ret:
            break
    
        frame_matches = corrected_df[corrected_df.Frame == frame_idx]
        frame_boxes = tracking_df[tracking_df.Frame == frame_idx]
        pose_row = pose_data_cleaned.loc[pose_data_cleaned["Frame"] == frame_idx]
    
        # Draw bounding boxes and IDs
        for _, box in frame_boxes.iterrows():
            x1, y1, x2, y2 = int(box.X1), int(box.Y1), int(box.X2), int(box.Y2)
            pid = int(box.Person_ID)
            color = colors.setdefault(pid, tuple(np.random.randint(0, 255, size=3).tolist()))
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
    
        # Draw skeletons and IDs
        if not pose_row.empty:
            individuals = pose_row.columns.get_level_values("individual").unique()
            individuals = [ind for ind in individuals if ind.startswith("idv_")]
            for ind in individuals:
                keypoints = {}
                for bp in bodyparts:
                    try:
                        x = float(pose_row[(ind, bp, "x")].values[0])
                        y = float(pose_row[(ind, bp, "y")].values[0])
                        if pd.notna(x) and pd.notna(y):
                            keypoints[bp] = (int(x), int(y))
                    except:
                        continue
    
                # Draw joints
                for bp, (x, y) in keypoints.items():
                    color = bodypart_colors.get(bp, (255, 255, 0))
                    cv2.circle(frame, (x, y), 5, color, -1)
    
                # Draw skeleton connections
                skeleton_connections = [
                    ("left_shoulder", "right_shoulder"), ("left_shoulder", "left_elbow"), ("left_elbow", "left_wrist"),
                    ("right_shoulder", "right_elbow"), ("right_elbow", "right_wrist"),
                    ("left_shoulder", "left_hip"), ("right_shoulder", "right_hip"),
                    ("left_hip", "right_hip"), ("left_hip", "left_knee"), ("left_knee", "left_ankle"),
                    ("right_hip", "right_knee"), ("right_knee", "right_ankle"),
                    ("nose", "left_eye"), ("nose", "right_eye"),
                    ("left_eye", "left_ear"), ("right_eye", "right_ear")
                ]
                for bp1, bp2 in skeleton_connections:
                    if bp1 in keypoints and bp2 in keypoints:
                        cv2.line(frame, keypoints[bp1], keypoints[bp2], (255, 255, 255), 2)
    
        # Draw corrected IDs
        for _, box in frame_boxes.iterrows():
            pid = int(box.Person_ID)
            x_center = int((box.X1 + box.X2) / 2)
            y_center = int((box.Y1 + box.Y2) / 2)
            color = colors.setdefault(pid, tuple(np.random.randint(0, 255, size=3).tolist()))
            cv2.putText(frame, f'ID {pid}', (x_center + 10, y_center - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 5)
    
        out_video.write(frame)
    
        progress_bar.update(1)
    
    cap.release()
    out_video.release()
    print("✅ Visual inspection video created!")




# === FINAL CORRECT MATCHED CSV (with likelihoods) ===

final_data = []

for idx, row in tqdm(final_output.iterrows(), total=len(final_output), desc="Creating Final Output"):
    frame_num = int(row["Frame"])
    box_id = int(row["Matched_Person_ID"])
    skel_id = row["Skeleton_ID"]

    # Prepare one row
    data = {
        "Frame": frame_num,
        "ID": box_id,
        "X1": row["X1"],
        "Y1": row["Y1"],
        "X2": row["X2"],
        "Y2": row["Y2"],
    }

    # Find skeleton number
    if isinstance(skel_id, str) and skel_id.startswith("idv_"):
        skel_num = skel_id.split("_")[1]
    else:
        continue  # skip if no skeleton match

    # Add bodyparts (x, y, likelihood)
    for part in target_bodyparts:
        x_col = f"idv_{skel_num}_{part}_x"
        y_col = f"idv_{skel_num}_{part}_y"
        conf_col = f"idv_{skel_num}_{part}_likelihood"
        
        data[f"{part}_x"] = row.get(x_col, np.nan)
        data[f"{part}_y"] = row.get(y_col, np.nan)
        data[f"{part}_conf"] = row.get(conf_col, np.nan)

    final_data.append(data)

# Create DataFrame
final_df = pd.DataFrame(final_data)

# Sort nicely
final_df = final_df.sort_values(["Frame", "ID"])

# Save
final_df.to_csv(OUTPUT_FOLDER / "final_corrected_flat_output_with_likelihood.csv", index=False)

print("✅ Corrected FINAL CSV saved with x, y, and likelihoods!")